In [15]:
#imports
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid

In [ ]:
#Parse Input
raw_text = ""
pdf_reader = PdfReader("Input/input.pdf")
for page in pdf_reader.pages:
    raw_text += page.extract_text()

#Split data
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
chunks = splitter.split_text(raw_text)
print(len(chunks))


In [ ]:
#Build input Dataframe
rows = []
for chunk in chunks:
    row = {
        "text": chunk,
        "chunk_id": uuid.uuid4().hex,
    }
    rows = rows + [row]
df = pd.DataFrame(rows)
print(df)

In [ ]:
#RUNNING THIS COSTS MONEY!!!!!!!!!!!!!!!
#----------------------------------------------------------
#Get the concepts
from graph_helper import get_concepts
from graph_helper import formatDF

generate_dataframes = False
if(generate_dataframes):
    concepts = df.apply(lambda row: get_concepts(row), axis=1)
    concepts = concepts.dropna()
    concepts = concepts.reset_index(drop=True)
    concept_list = np.concatenate(concepts).ravel().tolist()
    dfg1 = formatDF(concept_list)

    dfg1.to_csv("Output/graph.csv", sep="|", index=False)
    df.to_csv("Output/chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv("output/graph.csv", sep="|")
dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
 
print(dfg1.shape)
print(dfg1)

In [ ]:
df.apply(lambda x: print(x))